# *Fine-tuning* de *Large Language Models* (LLMs)

<p align="center">
  <a href="https://colab.research.google.com/github/auduvignac/llm-finetuning/blob/main/notebooks/project/finetuning-projet.ipynb" target="_blank">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Ouvrir dans Google Colab"/>
  </a>
</p>

Le but de ce projet est de réaliser le *fine-tuning* d'un LLM.


## Installation des bibliothèques/libraires requises

In [1]:
!wget -q https://raw.githubusercontent.com/auduvignac/llm-finetuning/refs/heads/main/setup_env.py -O setup_env.py
%run setup_env.py

⚡ Exécution sur Colab : vérification stricte des dépendances…
🔁 accelerate 1.10.0 ne satisfait pas '==1.8.1' → réinstallation : accelerate==1.8.1
⬇️ Manquant : bitsandbytes==0.46.0
✅ datasets 4.0.0 — OK
✅ matplotlib 3.10.0 — OK
✅ numpy 2.0.2 — OK
🔁 peft 0.17.0 ne satisfait pas '==0.15.0' → réinstallation : peft==0.15.0
✅ tabulate 0.9.0 — OK
✅ torch 2.8.0+cu126 — OK
✅ tqdm 4.67.1 — OK
✅ transformers 4.55.2 — OK
⬇️ Installation/réininstallation des paquets nécessaires…
 $ /usr/bin/python3 -m pip install accelerate==1.8.1
 $ /usr/bin/python3 -m pip install bitsandbytes==0.46.0
 $ /usr/bin/python3 -m pip install peft==0.15.0


In [3]:
%matplotlib inline
%config InlineBackend.figure_formats = ['svg']

import math

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers
from datasets import (
    load_dataset,
)
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from tabulate import (
    tabulate,
)
from torch.utils.data import (
    DataLoader,
)
from tqdm import (
    tqdm,
)

# Si le notebook est exécuté dans un environnement jupyter, la librairie
# ci-dessus peut être utilisée
from tqdm.notebook import (
    tqdm,
)
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    DistilBertConfig,
    DistilBertModel,
    DistilBertTokenizer,
)

# Utilisation d’un GPU avec CUDA lorsque disponible sur la machine d’exécution.
# L’utilisation d’un GPU pour l’apprentissage entraîne souvent d’énormes
# accélérations lors de l’entraînement.
# Voir https://developer.nvidia.com/cuda-downloads pour installer CUDA
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE

device(type='cpu')

In [ ]:
class LlamaFineTuner:
  def __init__(self, model_id = "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"):
    self.model_id = model_id
    self.bnb_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_quant_type="nf4",
      bnb_4bit_compute_dtype="float16",
      bnb_4bit_use_double_quant=False,
    )
    self.tokenizer = AutoTokenizer.from_pretrained(self.model_id)
    self.model = AutoModelForCausalLM.from_pretrained(
        self.model_id,
        quantization_config=self.bnb_config,
        device_map="auto"
    )

## Classification de textes avec les Transformers

Ce projet porte sur une tâche de **classification de textes** appliquée au **jeu de données IMDB** (analyse de sentiments ou *sentimental analysis*).  
Ce travail s'appuiera sur les **architectures de type encodeur**, en particulier l’un des modèles les plus connus : **BERT** (et sa variante légère **DistilBERT**).

## Présentation

Le projet consiste à utiliser la librairie `datasets` pour le chargement des données et les `tokenizers` pour le prétraitement des textes.

La librairie **Transformers** propose une API simple pour utiliser des modèles pré-entraînés tels que **BERT** ou **GPT**. Elle facilite leur téléchargement, leur ré-entraînement et leur intégration, tout en réduisant les coûts de calcul et en restant compatible avec **PyTorch, TensorFlow et JAX**.

## Objectif du projet

Dans le cadre de ce projet, l’expérimentation portera sur la librairie **Hugging Face** afin de :
- Charger et adapter un modèle de type **BERT** à une tâche de classification de textes ;
- Évaluer ses performances sur le dataset **IMDB** ;
- Analyser les résultats et discuter des choix réalisés (modèle, preprocessing, paramètres, etc.).

Le travail sera guidé par les interrogations suivantes :

- Bien que tous ces modèles reposent sur l'architecture **Transformer**, quelles en sont les spécificités ?
- Quel format d'entrée est attendu par le modèle ?
- Quels types de sorties génère-t-il ?
- Le modèle peut-il être utilisé tel quel ou doit-il être adapté à la tâche considérée ?

Ces questions constituent une part essentielle du travail quotidien d’un chercheur en NLP et seront examinées dans le cadre de ce projet de *fine-tuning*.

## Chargement du jeu d'entraînement

La phase initiale consiste à procéder au chargement du jeu d'entraînement au moyen de l'instruction suivante :
```python
dataset = load_dataset("scikit-learn/imdb", split="train")
```

In [ ]:
dataset = load_dataset("scikit-learn/imdb", split="train")

Après le chargement, il convient de procéder à l'affichage du jeu de données afin d'en examiner la structure.

In [ ]:
print(dataset)

L'exécution de la commande `print(dataset)` renvoie la description suivante du jeu de données :

```
Dataset({
    features: ['review', 'sentiment'],
    num_rows: 50000
})
```

Le corpus est constitué de 50 000 instances, chacune décrite par deux attributs : le texte de la critique (*review*) et son étiquette de polarité (*sentiment*).


## Préparation des entrées du modèle

Le format d'entrée attendu par **BERT** peut être considéré comme « sur-spécifié », notamment lorsqu'il s'agit de tâches ciblées telles que la *sequence classification*, le *word tagging* ou la *paraphrase detection*. Ce format repose sur plusieurs contraintes :

* l'ajout de *special tokens* au début et à la fin de chaque phrase ;
* l'application d'un *padding* et d'une troncature afin de ramener toutes les phrases à une longueur constante ;
* la distinction explicite entre *real tokens* et *padding tokens* au moyen de l'*attention mask*.


<p align="center">
<img src="https://raw.githubusercontent.com/auduvignac/llm-finetuning/97e2b676168167ed5ff624f1ad98589c63919d5d/figures/bert_encoding_process.png" width="600">
</p>

La figure ci-dessus illustre le processus de préparation et de traitement des séquences textuelles dans le modèle **BERT**.

1. **Tokenisation et ajout de tokens spéciaux**
   La séquence textuelle est d'abord segmentée en *tokens*. Deux *special tokens* sont ajoutés : `[CLS]`, placé en début de séquence et utilisé comme représentation globale, ainsi que `[SEP]`, placé en fin de séquence et jouant un rôle de séparateur.

2. **Normalisation de la longueur des séquences**
   Afin d'assurer une longueur uniforme entre les exemples, les séquences sont complétées par des *\[PAD] tokens* ou, le cas échéant, tronquées à une taille maximale prédéfinie (*MAX\_LEN*).

3. **Attention mask**
   Un vecteur binaire, appelé *attention mask*, est associé à chaque séquence. La valeur `1` indique un *real token* tandis que la valeur `0` correspond à un *padding token*. Ce mécanisme permet au modèle d'ignorer les positions de remplissage au cours de l'entraînement et de l'inférence.

4. **Propagation à travers les couches du Transformer**
   Les représentations vectorielles des tokens traversent successivement les différentes couches de l'encodeur Transformer (ici, 12 couches). Chaque couche applique un mécanisme d'auto-*attention*, permettant de capturer les dépendances contextuelles entre tokens.

5. **Production de la prédiction**
   À l'issue de la dernière couche, seule la représentation associée au token `[CLS]` est retenue. Celle-ci est transmise au classificateur, qui produit la prédiction finale (par exemple, la polarité d'une critique dans une tâche de *sentiment analysis*).

**Remarque :** Pour des raisons liées aux coûts de calcul, le modèle [DistilBERT](https://huggingface.co/docs/transformers/model_doc/distilbert) sera utilisé. Ce dernier constitue une version réduite d'environ 40 % par rapport à BERT, tout en conservant près de 95 % des performances du modèle original.

Afin de préparer les données textuelles pour le modèle, il est nécessaire d'instancier un *tokenizer*. Celui-ci a pour rôle de segmenter les phrases en unités élémentaires (*tokens*) et de les convertir en identifiants numériques exploitables par le modèle. Dans le cadre de ce projet, il sera fait usage du `DistilBertTokenizer` pré-entraîné, correspondant au modèle *distilbert-base-uncased*.

```python
tokenizer = DistilBertTokenizer.from_pretrained(
    "distilbert-base-uncased", do_lower_case=True
)
```


In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained(
    "distilbert-base-uncased", do_lower_case=True
)

L'étape suivante consiste à examiner la manière dont le *tokenizer* traite la séquence.

1. Définition d'un message exemple

In [ ]:
message = "hello my name is kevin"

2. *Tokenization* de la séquence

In [ ]:
tok = tokenizer.tokenize(message)
print("Tokens dans la séquence:", tok)

3. Encodage en identifiants numériques

In [ ]:
enc = tokenizer.encode(tok)

4. Mise en correspondance entre token IDs et tokens

In [ ]:
table = np.array(
    [
        enc,
        [tokenizer.ids_to_tokens[w] for w in enc],
    ]
).T

5. Affichage des résultats encodés

In [ ]:
print("Données d'entrée encodées:")
print(tabulate(table, headers=["Token IDs", "Tokens"], tablefmt="fancy_grid"))

Le tableau obtenu met en évidence la correspondance entre les *tokens* et leurs identifiants numériques.
Les *tokens* spéciaux `[CLS]` et `[SEP]` encadrent la séquence, tandis que chaque mot du texte est associé à un identifiant unique destiné au traitement par le modèle.

Les special tokens `[CLS]` et `[SEP]` sont ajoutés automatiquement par HuggingFace afin de structurer la séquence d'entrée pour le modèle BERT et ses variantes.

Le token `[CLS]`, inséré en début de séquence, constitue une représentation globale de la phrase. Sa sortie est utilisée par la couche de classification pour produire la prédiction finale (par exemple en *sentiment analysis*).

Le token `[SEP]`, placé en fin de séquence (ou comme séparateur entre deux phrases), sert à marquer les frontières de segments textuels. Il est essentiel dans les tâches nécessitant plusieurs entrées, comme la comparaison de paires de phrases (*natural language inference*, *paraphrase detection*).

Ainsi, ces *tokens* spéciaux garantissent une structuration normalisée des entrées, indispensable au fonctionnement du modèle.

## Prétraitement des données

Les opérations de prétraitement appliquées aux données suivent la méthodologie usuelle adoptée pour PyTorch et sont analogues à celles mises en œuvre précédemment.

La fonction `preprocessing_fn` définit les opérations de préparation appliquées à chaque exemple du corpus.
Le champ `review` est converti en une séquence d'identifiants numériques au moyen du *tokenizer*. L'encodage est effectué sans ajout de *special tokens*, avec troncature à une longueur maximale de 256 et sans application de *padding* ni génération de *attention mask*.
Par ailleurs, l'étiquette `sentiment` est transformée en valeur binaire : `0` pour les exemples négatifs et `1` pour les exemples positifs.

In [ ]:
def preprocessing_fn(x, tokenizer):
    """
    Prétraite un exemple du corpus en encodant le texte et en transformant
    l'étiquette.

    Paramètres
    ----------
    x : dict
        Exemple du corpus contenant au moins deux clés :
        - "review" (str) : le texte à encoder,
        - "sentiment" (str) : étiquette textuelle ("negative" ou "positive").
    tokenizer : transformers.PreTrainedTokenizer
        Tokenizer Hugging Face utilisé pour convertir le texte en identifiants numériques.

    Retour
    ------
    dict
        Dictionnaire enrichi contenant :
        - "input_ids" (List[int]) : séquence encodée de tokens numériques,
        - "labels" (int) : étiquette binaire (0 = négatif, 1 = positif).
    """
    x["input_ids"] = tokenizer.encode(
        x["review"],
        add_special_tokens=False,
        truncation=True,
        max_length=256,
        padding=False,
        return_attention_mask=False,
    )
    x["labels"] = 0 if x["sentiment"] == "negative" else 1
    return x

In [ ]:
n_samples = 2000  # nombre d'exemples d'entraînement

# opération de shuffle sur les données
dataset = dataset.shuffle()

# Selection de 2000 échantillons
splitted_dataset = dataset.select(range(n_samples))

# Tokenization du dataset
splitted_dataset = splitted_dataset.map(
    preprocessing_fn, fn_kwargs={"tokenizer": tokenizer}
)


# Suppression des colonnes inutiles
splitted_dataset = splitted_dataset.select_columns(["input_ids", "labels"])

# Séparation en données d'entraînement et de validation
splitted_dataset = splitted_dataset.train_test_split(test_size=0.2)

train_set = splitted_dataset["train"]
valid_set = splitted_dataset["test"]

In [ ]:
class DataCollator:
    """
    A data collator that pads input batches to a maximum length of 256 tokens.

    Args:
        tokenizer: The tokenizer to use for padding the input batch.

    Returns:
        Padded batch with tokenized inputs.
    """

    def __init__(self, tokenizer):
        """
        Initializes the DataCollator with the given tokenizer.

        Args:
            tokenizer: The tokenizer to use for padding the input batch.
        """
        self.tokenizer = tokenizer

    def __call__(self, batch):
        """
        Pads the input batch to a maximum length of 256 tokens using the
        provided tokenizer.

        Args:
            batch: The input batch to be padded.

        Returns:
            Padded batch with tokenized inputs.
        """
        return self.tokenizer.pad(
            batch, padding="longest", max_length=256, return_tensors="pt"
        )
        return features

In [ ]:
data_collator = DataCollator(tokenizer)

In [ ]:
batch_size = 4

train_dataloader = DataLoader(
    train_set, batch_size=batch_size, collate_fn=data_collator
)
valid_dataloader = DataLoader(
    valid_set, batch_size=batch_size, collate_fn=data_collator
)
n_valid = len(valid_set)
n_train = len(train_set)

Pour cette tâche, l'entraînement sera effectué à partir d'un modèle initialisé aléatoirement.

Récupération de la configuration de l'architecture

Dans la bibliothèque Hugging Face, les paramètres du modèle sont spécifiés par l'intermédiaire d'un fichier de configuration.

La configuration du modèle peut être récupérée grâce au code suivant :

In [ ]:
from transformers import DistilBertConfig

model_config = DistilBertConfig.from_pretrained("distilbert-base-uncased")
print(model_config)